In [1]:
import pandas as pd
import geopandas as gpd
import glob
import os

In [ ]:
def objects_count(input_polygons, directory):
    ''' 
    Данная функция подсчитывает плотность точек/процент перекрытия входным слоем относительно целевого полигона.
    '''
    polygons = gpd.read_file(input_polygons)
    gpkg_list = glob.glob(f"{directory}/*.gpkg")
    for gpkg in gpkg_list:
        name = os.path.basename(gpkg).split(".")[0]
        poi = gpd.read_file(gpkg)
        if poi.geometry.type[0] == "Point": 
            polygons[f"{name}, on km2"] = polygons.sjoin(poi, how='inner', op='intersects').groupby(level=0).size()
            polygons[f"{name}, on km2"] = polygons[f"{name}, on km2"].fillna(0)
            polygons[f"{name}, on km2"] = [int(num) for num in polygons[f"{name}, on km2"].tolist()]
            polygons["km2"] = polygons.to_crs(epsg=32435).geometry.area / 1000000
            polygons[f"{name}, on km2"] = [round(num / area, 1) for num, area in zip(polygons[f"{name}, on km2"], polygons["km2"])] 
            # Подсчитываем плотность точек внутри полигона
        elif poi.geometry.type[0] == "Polygon":
            polygons[f"{name}, overlap"] = polygons.to_crs(epsg=32435).geometry.intersection(poi.to_crs(epsg=32435).unary_union).area / polygons.to_crs(epsg=32435).geometry.area
            polygons[f"{name}, overlap"] = polygons[f"{name}, overlap"].apply(lambda x: round(x, 2)) # Подсчитываем перекрытие полигонами
        else:
            raise TypeError 
    polygons.drop(columns=["km2"]).to_file(f"{os.path.basename(input_polygons).split(".")[0]}_counted.gpkg")
    return polygons
        


In [67]:
objects_count("C:/Users/user/Desktop/Геофак учёба/1 МАГА/Пайтон у Энтина/Проект/Получение графа и изохрон/isochrones_geopackages/fifteen_minutes_iso.gpkg",
              "C:/Users/user/Desktop/Геофак учёба/1 МАГА/Пайтон у Энтина/Проект/Выгрузка POI")

C:\Users\user\AppData\Local\Temp\ipykernel_20520\3499132273.py:1: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  objects_count("C:/Users/user/Desktop/Геофак учёба/1 МАГА/Пайтон у Энтина/Проект/Получение графа и изохрон/isochrones_geopackages/fifteen_minutes_iso.gpkg",
C:\Users\user\AppData\Local\Temp\ipykernel_20520\3499132273.py:1: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  objects_count("C:/Users/user/Desktop/Геофак учёба/1 МАГА/Пайтон у Энтина/Проект/Получение графа и изохрон/isochrones_geopackages/fifteen_minutes_iso.gpkg",
C:\Users\user\AppData\Local\Temp\ipykernel_20520\3499132273.py:1: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  objects_count("C:/Users/user/Desktop/Геофак учёба/1 МАГА/Пайтон у Энтина/Проект

,name,walk_time,geometry,"food, on km2",km2,"greenery, overlap","leisure, on km2","shops, on km2","tourism, on km2"
0,Девяткино,15,"POLYGON ((30.43557 60.04412, 30.42973 60.04460...",44.3,2.165935,0.08,6.0,121.4,0.5
1,Гражданский проспект,15,"POLYGON ((30.40796 60.02760, 30.40489 60.02895...",14.7,2.518521,0.09,4.4,81.8,3.6
2,Академическая,15,"POLYGON ((30.39173 60.00419, 30.38917 60.00426...",25.8,2.632005,0.11,4.9,107.5,2.7
3,Политехническая,15,"POLYGON ((30.36599 59.99962, 30.36390 59.99966...",10.9,2.743177,0.23,5.1,15.3,2.6
4,Площадь Мужества,15,"POLYGON ((30.35875 59.99250, 30.35238 59.99381...",14.8,2.633557,0.21,4.6,26.2,3.4
...,...,...,...,...,...,...,...,...,...
67,Бухарестская,15,"POLYGON ((30.37424 59.87742, 30.37269 59.87743...",9.5,1.675502,0.14,2.4,33.4,6.0
68,Международная,15,"POLYGON ((30.37573 59.86133, 30.37371 59.86165...",12.0,2.588927,0.17,2.3,71.5,1.2
69,Проспект Славы,15,"POLYGON ((30.39919 59.84837, 30.38712 59.84928...",13.1,2.673197,0.28,3.0,53.5,6.0
70,Дунайская,15,"POLYGON ((30.42191 59.83074, 30.40611 59.83076...",5.9,2.523797,0.34,2.8,28.5,1.6
